In [4]:
import os
import pickle
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy.stats import pearsonr

In [5]:
def age_debiasing(predicted_ages, chronological_ages):
    
    predicted_ages = predicted_ages.to_numpy()
    chronological_ages = chronological_ages.to_numpy()
    
    ones = np.ones(len(chronological_ages))
    chronological_ages_squared = np.square(chronological_ages)
    chronological_ages_confounds = np.array([ones, chronological_ages])
    
    ones_predicted = np.ones(len(predicted_ages))
    predicted_ages_squared = np.square(predicted_ages)
    predicted_ages_confounds = np.array([ones_predicted, predicted_ages])
    
    predicted_ages = np.reshape(predicted_ages, (len(predicted_ages), 1) )
    beta = np.dot( np.linalg.pinv(chronological_ages_confounds.T), predicted_ages )

    corrected_predicted_ages = (predicted_ages.flatten() - beta[0])/beta[1]
    
    return corrected_predicted_ages

In [16]:
net1 = "1"


dfs = [
    net1,
]

names = [
    'Baseline',
]


dfs1, dfs2, dfs3 = [], [], []

dfs1 = dfs.copy()
# dfs2 = dfs.copy()
# dfs3 = dfs.copy()

for idx in range(len(dfs)):
    
    dfs1[idx] = '../../multi-age-mapper/predictions/AC1-' + dfs1[idx] + '_predictions/output_statistics_test.csv'
#     dfs2[idx] = '../predictions/AM3-' + dfs2[idx] + '_predictions/output_statistics_test.csv'
#     dfs3[idx] = '../predictions/AM4-' + dfs3[idx] + '_predictions/output_statistics_test.csv'
    
for idx in range(len(dfs)):
    dfs1[idx] = pd.read_csv(dfs1[idx])
#     dfs2[idx] = pd.read_csv(dfs2[idx])
#     dfs3[idx] = pd.read_csv(dfs3[idx])

In [17]:
dfs = dfs1

In [18]:
dfs[0]

,Unnamed: 0,target_age,output_age,age_delta,loss
0,24806531,67.670662,64.726204,-2.944458,8.669834
1,24237795,77.280936,76.979040,-0.301894,0.091140
2,24748274,58.301941,57.279945,-1.021995,1.044474
3,25463415,58.056694,60.786385,2.729691,7.451211
4,25313661,59.653082,57.004158,-2.648924,7.016799
...,...,...,...,...,...
5015,25475074,59.848516,60.156013,0.307498,0.094555
5016,22834218,60.501941,61.140970,0.639028,0.408356
5017,24442683,56.130708,54.246284,-1.884423,3.551051
5018,21157566,78.016553,76.624460,-1.392094,1.937926


In [19]:
# for idx in range(len(dfs)):
#     dfs[idx] = pd.merge(dfs1[idx], dfs2[idx], on='Unnamed: 0', how='inner')
#     dfs[idx] = pd.merge(dfs[idx], dfs3[idx], on='Unnamed: 0', how='inner')
    
#     dfs[idx].rename({
#         'target_age_x': 'target_age_original',
#         'output_age_x': 'output_age_1',
#         'age_delta_x': 'age_delta_1',
#         'loss_x': 'loss_1',
#         'target_age_y': 'target_age_2',
#         'output_age_y': 'output_age_2',
#         'age_delta_y': 'age_delta_2',
#         'loss_y': 'loss_2',
#         'target_age': 'target_age_3',
#         'output_age': 'output_age_3',
#         'age_delta': 'age_delta_3',
#         'loss': 'loss_3',
#     }, axis=1, inplace=True)
    
#     dfs[idx].drop(['target_age_2', 'target_age_3'], axis=1, inplace=True)
    
#     dfs[idx].rename({
#         'target_age_original': 'target_age',
#     }, axis=1, inplace=True)
    
#     dfs[idx]['mean_output_age'] = dfs[idx][['output_age_1', 'output_age_2', 'output_age_3']].mean(axis=1)
#     dfs[idx]['mean_age_delta'] = dfs[idx]['mean_output_age'] - dfs[idx]['target_age']

In [20]:
# for idx in range(len(dfs)):    
#     dfs[idx]['output_age_decon'] = age_debiasing(dfs[idx].mean_output_age, dfs[idx].target_age) 
#     dfs[idx]['age_delta_decon'] = dfs[idx].output_age_decon - dfs[idx].target_age

for idx in range(len(dfs)):    
    dfs[idx]['output_age_decon'] = age_debiasing(dfs[idx].output_age, dfs[idx].target_age) 
    dfs[idx]['age_delta_decon'] = dfs[idx].output_age_decon - dfs[idx].target_age

In [21]:
# print("{:13} : {:15}  |  {:13}  |  {:13}  |  {:13}".format('Modality', 'Run1', "Run2", "Run3", "MEAN"))
# print('-----------------------------------------------------------------------------------------------')

# for idx in range(len(dfs)):
#     print("{:13} : {:7.3f} ± {:.3f}  |  {:.3f} ± {:.3f}  |  {:.3f} ± {:.3f}  |  MEAN --- {:.3f} ± {:.3f}".format(
#         names[idx],
#         np.abs(dfs[idx].age_delta_1).mean(), np.abs(dfs[idx].age_delta_1).std()/np.sqrt(len(dfs[idx])),
#         np.abs(dfs[idx].age_delta_2).mean(), np.abs(dfs[idx].age_delta_2).std()/np.sqrt(len(dfs[idx])),
#         np.abs(dfs[idx].age_delta_3).mean(), np.abs(dfs[idx].age_delta_3).std()/np.sqrt(len(dfs[idx])),
#         np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
#     )
#          )

In [22]:
# print("{:13} : {:14}  |  {:13}  ".format('Modality', 'MEAN (MAE)', "MEAN DECONF"))
# print('-----------------------------------------------------------------------------------------------')

# for idx in range(len(dfs)):
#     print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f} ".format(
#         names[idx],
#         np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
#         np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
#     )
#          )

print("{:23} : {:7}  |  {:13}  ".format('Modality', 'MEAN (MAE)', "MEAN DECONF"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:23} :  {:.3f}  |   {:.3f}  ".format(
        names[idx],
        np.abs(dfs[idx].age_delta).mean(), #np.abs(dfs[idx].age_delta).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_decon).mean(), # np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
    )
         )
    
# T1 NonLinear:    2.858 | 2.994
# T2 NonLinear:    2.707 | 3.297


Modality                : MEAN (MAE)  |  MEAN DECONF    
-----------------------------------------------------------------------------------------------
Baseline                :  2.487  |   2.742  


In [23]:
# Modality                : MEAN (MAE)      |  MEAN DECONF    
# -----------------------------------------------------------------------------------------------
# InputFusion             :  2.633 ± 0.063  |   2.906 ± 0.071 
# InputFusion_Filter      :  2.829 ± 0.069  |   3.267 ± 0.077 
# LayerFusion9600         :  2.794 ± 0.063  |   3.087 ± 0.073


# mae_weighting = 37.137671232876755 # for female subjects
# # mae_weighting = 36.28150684931529 # for male subjects


# print("{:13} : {:14}  |  {:15}  |  {:7}  |  {:13} ".format('Modality', 'MAE', "RMSE", 
#                                                             "CORR (R)", "MAE weighted"))
# print('-----------------------------------------------------------------------------------------------')

# for idx in range(len(dfs)):
#     print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f}   |   {:5.3f} ".format(
#         names[idx],
#         np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
#         np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age)).mean()), np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age)).std()/np.sqrt(len(dfs[idx]))),
#         pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0],
#         np.abs(dfs[idx].mean_age_delta).mean() / mae_weighting,
#     )
#          )


In [24]:
# maes=[]
# prs = []
# for idx in range(len(dfs)):
#     mae = np.abs(dfs[idx].mean_age_delta).mean()
#     maes.append(mae)
#     pr = pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0]
#     prs.append(pr)
    
# print(pearsonr(maes, prs)[0])

In [25]:
# print("{:13} : {:14}  |  {:15}  |  {:13} ".format('Modality', 'MAE (DECON)', "RMSE (DECON)", "PR (DECON)"))
# print('-----------------------------------------------------------------------------------------------')

# for idx in range(len(dfs)):
#     print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f} ".format(
#         names[idx],
#         np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
#         np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean()), np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).std()/np.sqrt(len(dfs[idx]))),
#         pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
#     )
#          )

# Saving Dataframe for Later Use

In [26]:
d = {'modality':names, 'dataframe':dfs}
df = pd.DataFrame(data=d)

In [27]:
df['dataframe'].iloc[0]

,Unnamed: 0,target_age,output_age,age_delta,loss,output_age_decon,age_delta_decon
0,24806531,67.670662,64.726204,-2.944458,8.669834,64.831357,-2.839305
1,24237795,77.280936,76.979040,-0.301894,0.091140,79.231294,1.950358
2,24748274,58.301941,57.279945,-1.021995,1.044474,56.080267,-2.221673
3,25463415,58.056694,60.786385,2.729691,7.451211,60.201151,2.144457
4,25313661,59.653082,57.004158,-2.648924,7.016799,55.756153,-3.896929
...,...,...,...,...,...,...,...
5015,25475074,59.848516,60.156013,0.307498,0.094555,59.460317,-0.388198
5016,22834218,60.501941,61.140970,0.639028,0.408356,60.617871,0.115931
5017,24442683,56.130708,54.246284,-1.884423,3.551051,52.515009,-3.615699
5018,21157566,78.016553,76.624460,-1.392094,1.937926,78.814580,0.798028


In [28]:
df

,modality,dataframe
0,Baseline,Unnamed: 0 target_age output_age age_...


In [29]:
type(df.dataframe.iloc[0])

pandas.core.frame.DataFrame

In [30]:
df.iloc[0].dataframe

,Unnamed: 0,target_age,output_age,age_delta,loss,output_age_decon,age_delta_decon
0,24806531,67.670662,64.726204,-2.944458,8.669834,64.831357,-2.839305
1,24237795,77.280936,76.979040,-0.301894,0.091140,79.231294,1.950358
2,24748274,58.301941,57.279945,-1.021995,1.044474,56.080267,-2.221673
3,25463415,58.056694,60.786385,2.729691,7.451211,60.201151,2.144457
4,25313661,59.653082,57.004158,-2.648924,7.016799,55.756153,-3.896929
...,...,...,...,...,...,...,...
5015,25475074,59.848516,60.156013,0.307498,0.094555,59.460317,-0.388198
5016,22834218,60.501941,61.140970,0.639028,0.408356,60.617871,0.115931
5017,24442683,56.130708,54.246284,-1.884423,3.551051,52.515009,-3.615699
5018,21157566,78.016553,76.624460,-1.392094,1.937926,78.814580,0.798028


In [31]:
df.iloc[0].modality

'Baseline'

In [21]:
type(df.iloc[0].dataframe)

pandas.core.frame.DataFrame

In [32]:
df.to_pickle('female_test_baseline.pkl', protocol = 4)

In [23]:
# df2 = pd.read_pickle('female_test.pkl')

In [33]:
# df2

In [34]:
# df2.iloc[0].dataframe

In [35]:
# df2.iloc[0].modality

In [36]:
# type(df2.iloc[0].dataframe)

# BarPlots

In [37]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.mean_output_age.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.array(df_test)

# fig = plt.figure(figsize=(16, 7.2))
# plt.boxplot(array);
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("Predicted Age Distributions")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);

In [38]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.mean_age_delta.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.array(df_test)

# fig = plt.figure(figsize=(16, 7.2))
# plt.boxplot(array);
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("Predicted Age Delta Distributions")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);

In [39]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.output_age_decon.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.array(df_test)

# fig = plt.figure(figsize=(16, 7.2))
# plt.boxplot(array);
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("Predicted Debiased Age Distributions")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);

In [40]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.age_delta_decon.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.array(df_test)

# fig = plt.figure(figsize=(16, 7.2))
# plt.boxplot(array);
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("Predicted Debiased Age Deltas Distributions")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);

In [41]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.age_delta_decon.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.abs(np.array(df_test))

# fig = plt.figure(figsize=(16, 7.2))
# plt.boxplot(array);
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("Predicted Absolute Debiased Age Deltas Distributions")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);

In [42]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.mean_age_delta.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.array(df_test)

# fig = plt.figure(figsize=(16, 7.2))
# # plt.scatter(np.arange(1, len(array), 1), array);
# plt.scatter(np.arange(1, array.shape[1]+1), abs(array).mean(axis=0))
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("MAE")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axhline(y=6.253, color='r', linestyle='dashed')

# Test what happens if debiasing before deconfounding

In [43]:
# dfs[0]

In [44]:
# for idx in range(len(dfs)):
#     dfs[idx]['output_age_decon_1'] = age_debiasing(dfs[idx].output_age_1, dfs[idx].target_age)
#     dfs[idx]['output_age_decon_2'] = age_debiasing(dfs[idx].output_age_2, dfs[idx].target_age) 
#     dfs[idx]['output_age_decon_3'] = age_debiasing(dfs[idx].output_age_3, dfs[idx].target_age) 
#     dfs[idx]['mean_output_age_decon'] = dfs[idx][['output_age_decon_1', 'output_age_decon_2', 'output_age_decon_3']].mean(axis=1)
#     dfs[idx]['mean_age_delta_decon'] = dfs[idx]['mean_output_age_decon'] - dfs[idx]['target_age']

In [45]:
# dfs[0]

In [46]:
# print("{:13} : {:14}  |  {:14}  |  {:13}  |   {:13}  |  {:}  |  {:} ".format('Modality', 'MAE (DECON)', 'MAE (DECON B)', "MSE (DECON)", "MSE (DECON B)",  "PR (DECON)", "PR (DECON B)"))
# print('-----------------------------------------------------------------------------------------------')

# for idx in range(len(dfs)):
#     print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f}   |   {:.3f} ".format(
#         names[idx],
        
#         np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
#         np.abs(dfs[idx].mean_age_delta_decon).mean(), np.abs(dfs[idx].mean_age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        
#         np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).std()/np.sqrt(len(dfs[idx])),
#         np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).std()/np.sqrt(len(dfs[idx])),
        
#         pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
#         pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age_decon)[0],
#     )
#          )